In [7]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import wandb
wandb.login()
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

# set seed for reproducibility
random_seed = 1

In [8]:
df = pd.read_csv('fred_230718.csv', index_col='Date', parse_dates=True)
df = df.asfreq('QS')
earliest_date = '1963-01-01'
latest_date = '2021-10-01'
# # filter df index to be between earliest_date and latest_date
df = df.loc[(df.index >= earliest_date) & (df.index <= latest_date)]
df.dropna(axis=1, inplace=True)
print(f'feature count: {len(df.columns) - 1}')
print(f'row count: {len(df)}')
df.head()

feature count: 25
row count: 236


,GDP,GDPC1,GDPPOT,CPIAUCSL,CPILFESL,GDPDEF,M1V,M2V,DFF,UNRATE,...,MANEMP,DSPIC96,PCE,PCEDG,PSAVERT,DSPI,INDPRO,HOUST,GPDI,MSPUS
Date,,,,,,,,,,,,,,,,,,,,,
1963-01-01,621.672,3628.306,3662.738125,30.44,31.5,17.134,4.178,1.690,3.00,5.7,...,15545.0,2541.1,374.4,53.1,10.9,430.0,26.0448,1244.0,99.689,17800.0
1963-04-01,629.752,3669.020,3701.698767,30.48,31.7,17.164,4.194,1.675,3.00,5.7,...,15602.0,2547.1,376.4,53.2,10.7,431.1,26.7473,1689.0,101.650,18000.0
1963-07-01,644.444,3749.681,3741.388301,30.69,31.8,17.187,4.248,1.680,3.00,5.6,...,15646.0,2572.6,384.4,55.5,10.1,438.0,27.0445,1614.0,104.612,17900.0
1963-10-01,653.938,3774.264,3781.880559,30.75,32.0,17.326,4.269,1.672,3.50,5.5,...,15714.0,2617.3,386.0,54.2,11.5,447.0,27.5578,1779.0,107.189,18500.0
1964-01-01,669.822,3853.835,3822.450115,30.94,32.2,17.381,4.345,1.685,3.25,5.6,...,15715.0,2652.8,396.8,57.9,10.7,455.3,27.8820,1603.0,110.474,18500.0


set target and create, train, validate, and test datasets and then scale and transform them so they will work better with the neural network

In [9]:
target = 'MSPUS'

drop_cols = ['GDPC1', 'GDPPOT', 'CPIAUCSL', 'CPILFESL', 'GDPDEF'
            , 'M2V'
            , 'DSPIC96', 'PCE', 'PCEDG', 'DSPI', 'INDPRO', 'GPDI']
df.drop(columns=drop_cols, inplace=True)

y = df[target]
X = df.drop(columns=[target]).shift(1).dropna()
y = y.loc[X.index] # Make sure y and X have the same rows after dropna

# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=random_seed)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)  # validation data should also be scaled
X_test_scaled = scaler.transform(X_test)

# Log-transform the target variable
y_train_log = np.log1p(y_train)
y_valid_log = np.log1p(y_valid)

In [10]:
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

def train_model(X_train, y_train, 
                X_valid, y_valid,
                layer_sizes=[100, 100], 
                activation="relu", 
                kernel_initializer="he_normal", 
                loss='mse',
                learning_rate=0.001, 
                epochs=100,
                batch_norm=False,
                l1_l2=False,
                l1=.01,
                l2=.01,
                metrics=['mse'],
                wandb = 'y'):

    # Create a sequential model
    model = tf.keras.Sequential()

    # Add batch normalization and dense layers according to the layer_sizes
    for size in layer_sizes:
        if batch_norm:
            model.add(tf.keras.layers.BatchNormalization())
        if l1_l2:
            model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer
                                            , kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
        else:
            model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer))

    # Add a final Dense layer with no activation
    model.add(tf.keras.layers.Dense(1))

    # Create the optimizer with the custom learning rate
    sgd = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    model.compile(loss=loss, optimizer=sgd, metrics=metrics)
    
    if wandb == 'y':
        # Add WandbMetricsLogger to log metrics and WandbModelCheckpoint to log model checkpoints
        wandb_callbacks = [
            WandbMetricsLogger(),
            # WandbModelCheckpoint(filepath="my_model_{epoch:02d}"),
            # WandbModelCheckpoint(filepath="my_model_best", save_best_only=True, monitor='val_loss'),
        ]

        # Train the model using the scaled data
        model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid), callbacks=wandb_callbacks)
    else:
        # Train the model using the scaled data
        model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid))
    
    return model

In [11]:
sweep_config = {
    'method': 'bayes',  # can be grid, random, or bayes
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 0.0001,
            'max': 0.01
        },
        'activation': {
            'values': ['relu']
        }, 
        'kernel_initializer': {
            'values': ['he_normal']
        }, 
        'loss': {
            'values': ['mse']
        }, 
        # 'activation': {
        #     'values': ['relu', 'tanh', 'sigmoid', 'elu', 'selu', 'softplus']  # more activation functions
        # }, 
        # 'kernel_initializer': {
        #     'values': ['he_normal', 'glorot_uniform', 'glorot_normal', 'lecun_normal']  # more initializers
        # }, 
        # 'loss': {
        #     'values': ['mse', 'mae', 'logcosh', 'huber']  # more loss functions
        # },
        'epoch': {
            'values': [100]
        }, 
        # 'epoch': {
        #     'min': 10,
        #     'max': 500
        # },
        'batch_norm': {
            'values': [True, False]
        }, 
        'l1_l2': {
            'values': [True]
        },        
        'l1': {
            'min': 0.0001,
            'max': 0.5
        },
        'l2': {
            'min': 0.0001,
            'max': 0.5
        },
        'metrics': {
            'values': ['mae']
        }
        # 'metrics': {
        #     'values': ['mae', 'mse', 'mape', 'msle']  # adding more metrics
        # }
    }
}

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model = train_model(
            X_train=X_train_scaled, y_train=y_train_log,
            X_valid=X_test_scaled, y_valid=y_valid_log,
            activation=config.activation, kernel_initializer=config.kernel_initializer, 
            loss=config.loss, learning_rate=config.learning_rate, 
            epochs=config.epoch, batch_norm=config.batch_norm, 
            l1_l2=config.l1_l2, l1=config.l1, l2=config.l2, 
            metrics=config.metrics)

sweep_id = wandb.sweep(sweep_config, project="fred_dnn_sweep")
wandb.agent(sweep_id, train, count=100)

Create sweep with ID: azovk6yv
Sweep URL: https://wandb.ai/grantbell/fred_dnn_sweep/sweeps/azovk6yv


wandb: Agent Starting Run: vcaadsj6 with config:
wandb: 	activation: relu
wandb: 	batch_norm: False
wandb: 	epoch: 100
wandb: 	kernel_initializer: he_normal
wandb: 	l1: 0.1669997122667422
wandb: 	l1_l2: True
wandb: 	l2: 0.319413580861995
wandb: 	learning_rate: 0.0044572913522162435
wandb: 	loss: mse
wandb: 	metrics: mae
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
5/5 [==============================] - 1s 40ms/step - loss: 418.2718 - mae: 4.4651 - val_loss: 378.8135 - val_mae: 2.1933
Epoch 2/100
5/5 [==============================] - 0s 13ms/step - loss: 370.7211 - mae: 1.8459 - val_loss: 359.7202 - val_mae: 1.9256
Epoch 3/100
5/5 [==============================] - 0s 14ms/step - loss: 351.3892 - mae: 1.3141 - val_loss: 343.4414 - val_mae: 2.0155
Epoch 4/100
5/5 [==============================] - 0s 13ms/step - loss: 334.4197 - mae: 1.1018 - val_loss: 326.7636 - val_mae: 1.8005
Epoch 5/100
5/5 [==============================] - 0s 12ms/step - loss: 318.3742 - mae: 0.9496 - val_loss: 311.0551 - val_mae: 1.7157
Epoch 6/100
5/5 [==============================] - 0s 13ms/step - loss: 303.0534 - mae: 0.8400 - val_loss: 296.2177 - val_mae: 1.6551
Epoch 7/100
5/5 [==============================] - 0s 11ms/step - loss: 288.4676 - mae: 0.7681 - val_loss: 283.1754 - val_mae: 1.9674
Epoch 8/100
5/5 [==============================] - 0s 11ms/ste

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/mae,█▃▂▂▂▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▇▆▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_mae,█▇▅▅▅▅▄▄▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch/epoch,99
epoch/learning_rate,0.00446
epoch/loss,1.60642
epoch/mae,0.2842
epoch/val_loss,2.63928


wandb: Ctrl + C detected. Stopping sweep.


the best epoch/val_mae is run 94
https://wandb.ai/grantbell/fred_dnn_sweep/runs/qgc616q6?workspace=user-grantbell

In [13]:
def evaluate_model(model, X_test, y_test, log_target=False):
    # When predicting, transform the predictions back
    y_pred = model.predict(X_test)
    if log_target:
        y_pred = np.expm1(y_pred).flatten()  # inverse of np.log1p(), make it 1D


    # compute the RMSE on the original scale
    mse = np.mean(tf.keras.losses.MSE(y_test, y_pred))
    print('Test set MSE:', mse)
    rmse = np.sqrt(mse)
    print('Test set RMSE:', rmse)
    rmspe = (np.sqrt(np.mean(np.square((y_test - y_pred) / y_test)))) * 100
    print('Test set RMSPE (%):', rmspe)
    
print('default')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log
                    , wandb='n')
evaluate_model(model, X_test_scaled, y_test, log_target=True)
print('run 94')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log
                    , wandb='n'
                    , l1 = .2484
                    , l1_l2=True
                    , l2 = .3598
                    , learning_rate=.00264)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

default
Epoch 1/100
5/5 [==============================] - 0s 35ms/step - loss: 112.5749 - mse: 112.5749 - val_loss: 46.4741 - val_mse: 46.4741
Epoch 2/100
5/5 [==============================] - 0s 10ms/step - loss: 27.5989 - mse: 27.5989 - val_loss: 15.6532 - val_mse: 15.6532
Epoch 3/100
5/5 [==============================] - 0s 10ms/step - loss: 11.1649 - mse: 11.1649 - val_loss: 11.5061 - val_mse: 11.5061
Epoch 4/100
5/5 [==============================] - 0s 12ms/step - loss: 7.6305 - mse: 7.6305 - val_loss: 9.3956 - val_mse: 9.3956
Epoch 5/100
5/5 [==============================] - 0s 10ms/step - loss: 5.9149 - mse: 5.9149 - val_loss: 8.8249 - val_mse: 8.8249
Epoch 6/100
5/5 [==============================] - 0s 9ms/step - loss: 4.8155 - mse: 4.8155 - val_loss: 8.2504 - val_mse: 8.2504
Epoch 7/100
5/5 [==============================] - 0s 10ms/step - loss: 4.1224 - mse: 4.1224 - val_loss: 7.9052 - val_mse: 7.9052
Epoch 8/100
5/5 [==============================] - 0s 10ms/step - los

next things to try
- shuffle within the train function so the train, test and validate sets are different each time?
- different number and size of layers
- different lags (e.i 1 year instead of one quarter) and/or combinations of lags
- different static number or make dynamic of epochs